In [1]:
from pytube import YouTube
from tqdm import tqdm
import cv2
from matplotlib import pyplot as plt
import pytesseract
import numpy as np
from PIL import Image

In [4]:
# highlights
yt = YouTube('https://www.youtube.com/watch?v=HF6wnXp3rsE')

stream = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()

pbar = tqdm(total=stream.filesize)
def progress_fn(self, chunk, *_):
    pbar.update(len(chunk))
yt.register_on_progress_callback(progress_fn)    

stream.download(output_path=".", filename="source_high_res_highlights") 

100%|█████████▉| 131235840/131434882 [00:53<00:00, 2122418.44it/s]

'./source_high_res_highlights.mp4'

100%|██████████| 131434882/131434882 [01:10<00:00, 2122418.44it/s]

In [14]:
# Crop image to get timestamp

def get_num_from_img(crop_img):
    plt.figure()
#     plt.subplot(161),plt.imshow(crop_img),plt.title('Original')

    crop_img = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)

    flip_img = 255 - crop_img

#     plt.subplot(162),plt.imshow(flip_img),plt.title('Flipped')

    # ret, thresh = cv2.threshold(flip_img, 127, 255, cv2.THRESH_BINARY) 
    ret, thresh = cv2.threshold(flip_img, 170, 255, cv2.THRESH_BINARY) 
#     plt.subplot(163),plt.imshow(thresh),plt.title('Threshold')


    upscale_img = cv2.resize(thresh, (thresh.shape[1] * 20, thresh.shape[0] * 20), interpolation = cv2.INTER_AREA) 
#     plt.subplot(164),plt.imshow(upscale_img),plt.title('Upscaled')

    blur = cv2.blur(upscale_img,(25,25))

    pil_img = Image.fromarray(blur)
    text = pytesseract.image_to_string(pil_img,  config='outputbase digits')
    
#     plt.subplot(165),plt.imshow(blur),plt.title('Blur ' + text)
    
    kernel_sharpening = np.array([[-1,-1,-1], 
                                  [-1, 9,-1],
                                  [-1,-1,-1]])
    
    sharpened = cv2.filter2D(blur, -1, kernel_sharpening)
    pil_img = Image.fromarray(sharpened)
    if text == "":
        text = pytesseract.image_to_string(pil_img,  config='outputbase digits')
#     plt.subplot(166),plt.imshow(sharpened),plt.title('Sharpened ' + text)

    
    pil_img.save("test.jpg", "JPEG")
    return text

def get_timestamp_from_img(img):
    height, width, chan = img.shape

    crop_img = img[int(height * 0.068): int(height * 0.1), int(width * 0.485) : int(width * 0.52)]
    crop_img_left = img[int(height * 0.07): int(height * 0.095), int(width * 0.49) : int(width * 0.503)]
#     plt.imshow(crop_img_left)
#     plt.figure()
    
    crop_img_left = np.pad(crop_img_left, [(5, 5), (5, 5), (0,0)], mode='constant')
#     plt.imshow(crop_img_left)
#     plt.figure()

    crop_img_right = img[int(height * 0.07): int(height * 0.09), int(width * 0.504) : int(width * 0.515)]
#     plt.imshow(crop_img_right)
#     plt.figure()
    
    crop_img_right = np.pad(crop_img_right, [(5, 5), (5, 5), (0,0)], mode='constant')
#     plt.imshow(crop_img_right)
#     plt.figure()
    
    
    return (get_num_from_img(crop_img_left),
            get_num_from_img(crop_img_right))

none_count = 0
for i in range(36):
    name = ('frames/frame%d.jpg' % (i * 40 + 580))
    print(name)
    img = cv2.imread(name)
    (left, right) = get_timestamp_from_img(img)
    print (left, right)
    if left == "":
        none_count += 1
    if right == "":
        none_count += 1
        
print(none_count)
print(((i * 2) - none_count) / (i * 2))


frames/frame580.jpg
07 47
frames/frame620.jpg
07 
frames/frame660.jpg
07 50
frames/frame700.jpg
07 51
frames/frame740.jpg
07 53
frames/frame780.jpg
07 54
frames/frame820.jpg
10 56
frames/frame860.jpg
 36
frames/frame900.jpg
 38
frames/frame940.jpg
10 39
frames/frame980.jpg


/home/jerry/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  after removing the cwd from sys.path.
/home/jerry/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  after removing the cwd from sys.path.
/home/jerry/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed 

10 41
frames/frame1020.jpg


/home/jerry/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  after removing the cwd from sys.path.


10 2
frames/frame1060.jpg


...

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

In [12]:
vidcap = cv2.VideoCapture('source_high_res_highlights.mp4')
success,image = vidcap.read()
count = 0
while success:
  if count % 20 == 0:
      cv2.imwrite("frames/frame%d.jpg" % count, image)     # save frame as JPEG file      
  success,image = vidcap.read()
  count += 1


KeyboardInterrupt: 